In [1]:
!pip install git+https://github.com/huggingface/peft.git

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-wi7yfr8i
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-wi7yfr8i
  Resolved https://github.com/huggingface/peft.git to commit 749b92456218f7dddc8f7a9aa27a41815b3d6c2e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for peft: filename=peft-0.13.3.dev0-py3-none-any.whl size=336362 sha256=84e2f4d648040c94eea01e74e79c3009f27abffcc4c16344add8c5e895b15143
  Stored in directory: /tmp/pip-ephem-wheel-cache-2aa0mf3e/wheels/d7/c7/de/1368fac8590e1b103ddc2ec2a28ad51d83aded1a3830e8a087
Successfully built peft


In [2]:
!pip show peft


Name: peft
Version: 0.13.3.dev0
Summary: Parameter-Efficient Fine-Tuning (PEFT)
Home-page: https://github.com/huggingface/peft
Author: The HuggingFace team
Author-email: benjamin@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: accelerate, huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch, tqdm, transformers
Required-by: 


In [3]:
!pip install transformers datasets peft accelerate


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.6 MB/s eta 0:00:00


In [13]:
from transformers import BertTokenizer, BertForSequenceClassification
from peft import LoraConfig, get_peft_model
import torch
from datasets import load_dataset
from transformers import Trainer, TrainingArguments
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import numpy as np


# Load pre-trained BERT tokenizer and model for classification
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Load a small portion of the dataset (for demonstration purposes)
dataset = load_dataset("imdb", split="train[:1%]")

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Split the tokenized dataset into train and eval datasets (80-20 split)
train_size = int(0.85 * len(tokenized_dataset))
train_dataset = tokenized_dataset.select(range(train_size))
eval_dataset = tokenized_dataset.select(range(train_size, len(tokenized_dataset)))

# LoRA Adapter configuration
lora_config = LoraConfig(
    r=8,              # Low-rank dimension
    lora_alpha=32,     # Scaling factor
    lora_dropout=0.1,  # Dropout probability for Lora layers
    task_type="SEQ_CLS"  # Task type: Sequence Classification
)


# Add LoRA to the BERT model using PEFT
model = get_peft_model(model, lora_config)

# Training arguments
training_args = {
    'output_dir': './results',  # Save checkpoint directory
    'evaluation_strategy': 'epoch',  # Evaluate after every epoch
    'learning_rate': 2e-5,
    'per_device_train_batch_size': 16,
    'per_device_eval_batch_size': 16,
    'num_train_epochs': 1,
    'weight_decay': 0.01,
}

training_arguments = TrainingArguments(**training_args)

# Compute metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)

    # Compute precision, recall, f1, and accuracy
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)

    return {
        'accuracy': acc,  # Accuracy
        'precision': precision,  # Precision
        'recall': recall,  # Recall
        'f1': f1  # F1 Score
    }

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics  # Pass the compute_metrics function
)

# Fine-tune the model
trainer.train() #Takes 4-6 minutes in colab

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.826845,0.000000,0.000000,0.000000,0.000000


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TrainOutput(global_step=14, training_loss=0.8150358881269183, metrics={'train_runtime': 203.144, 'train_samples_per_second': 1.044, 'train_steps_per_second': 0.069, 'total_flos': 13993152737280.0, 'train_loss': 0.8150358881269183, 'epoch': 1.0})

In [16]:
# Evaluate the model on the evaluation dataset
eval_results = trainer.evaluate() # in colab takes 1-2 minutes

# Print evaluation metrics
print(f"Evaluation Accuracy: {eval_results['eval_accuracy']}")
print(f"Evaluation Precision: {eval_results['eval_precision']}")
print(f"Evaluation Recall: {eval_results['eval_recall']}")
print(f"Evaluation F1 Score: {eval_results['eval_f1']}")



Evaluation Accuracy: 0.0
Evaluation Precision: 0.0
Evaluation Recall: 0.0
Evaluation F1 Score: 0.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [15]:

# Save the fine-tuned model and tokenizer
trainer.save_model("./fine_tuned_lora_model")
tokenizer.save_pretrained("./fine_tuned_lora_model")

# Reload the saved model and tokenizer for inference later
from transformers import BertForSequenceClassification, BertTokenizer

loaded_model = BertForSequenceClassification.from_pretrained("./fine_tuned_lora_model")
loaded_tokenizer = BertTokenizer.from_pretrained("./fine_tuned_lora_model")

# Test the reloaded model with a sample input
sample_text = "This movie was absolutely fantastic! The plot was great."
inputs = loaded_tokenizer(sample_text, return_tensors="pt", padding="max_length", truncation=True, max_length=128)
outputs = loaded_model(**inputs)

# Get prediction
predicted_class = torch.argmax(outputs.logits, dim=1).item()
class_labels = {0: "negative", 1: "positive"}  # Define your label mapping
predicted_class_label = class_labels[predicted_class]

print(f"Predicted class: {predicted_class_label}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Predicted class: positive
